## Workspace 정보 가져오기

In [ ]:
from azureml.core import Workspace
ws = Workspace.from_config()   # workspace의 setting 가져오기
print(
    'Workspace name\t: ' + 'XXXXXXXXXXXXXXXXXX',
    # 'Workspace name\t: ' + ws.name,
    '\nAzure region\t: ' + 'XXXXXXXXXXXXXXXXXX',
    # '\nAzure region\t: ' + ws.location,
    '\nSubscription ID\t: ' + 'XXXXXXXXXXXXXXXXXX',
    # '\nSubscription ID\t: ' + ws.subscription_id,
    '\nResource Group\t: ' + 'XXXXXXXXXXXXXXXXXX'
    # '\nResource Group\t: ' + ws.resource_group
)

## 실험 공간 생성

In [ ]:
from azureml.core import Experiment
experiment = Experiment(workspace= ws, name= 'diabetes_experiment') # workspace: 위에 설정한 workspace 지정, name: 실험의 이름

### Diabetes Dataset 가져오기

In [ ]:
from azureml.opendatasets import Diabetes
from sklearn.model_selection import train_test_split

x_df = Diabetes.get_tabular_dataset().to_pandas_dataframe().dropna()
y_df = x_df.pop('Y')    # x_df에서 ['Y']는 pop되고 y_df에 저장된다

# Split trainset & testset 
X_train, X_test, y_train, y_test = train_test_split(x_df, y_df, test_size= 0.2, random_state= 66)

## Model 불러오기 (Ridge)

In [ ]:
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error
from sklearn.externals import joblib
import math

alphas = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]

for alpha in alphas:
    run = experiment.start_logging()  # logging 기록 남기기
    run.log('alpha', alpha)

    model = Ridge(alpha= alpha)
    model.fit(X_train, y_train)

    y_pred = model.predict(X_test)

    # 루트: math.sqrt
    rmse = math.sqrt(mean_squared_error(y_test, y_pred) )    
    run.log('rmse: ', rmse)   # log에 남김
    
    #모델 이름 설정
    model_name = 'model_alpha_{}.pkl'.format(str(alpha))  
    filename = 'outputs/{}'.format(model_name)

    # 모델 저장
    joblib.dump(value= model, filename= filename)

    # 시스템에 올려놓기
    run.upload_file(name= model_name, path_or_stream= filename) # Azure ML Service에 업로드
    run.complete() # 작업종료

    print(f'{alpha} experiment completed')